In [1]:
from transformers import GPT2LMHeadModel
import pandas as pd 
import tqdm as tqdm
from transformers import GPT2Tokenizer
import pyterrier as pt 
# pt.started()
pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])
# pt.init()

/home/wzm289/miniconda3/envs/ACL/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-20 00:01:11.799376: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 00:01:14.574605: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-20 00:01:14.574742: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugi

In [2]:
# load tokenizer

tokenizer = GPT2Tokenizer.from_pretrained(
    "/projects/futhark1/data/wzm289/code/RLSeq2SeqPytorch/models/tokenizer"
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"


In [38]:
# fout = open("result/gpt_predict.txt", "w")

# def test():
#     model = GPT2LMHeadModel.from_pretrained("result").cuda()
#     model.resize_token_embeddings(len(tokenizer))
#     model.eval()

#     df = pd.read_csv("/projects/futhark1/data/wzm289/code/RLSeq2SeqPytorch/scifact/test.source", names=["source"], sep="\t")
#     print(df)
#     for text in df["source"].to_list():
#         prompt = text

#         # note that we only want to make the parameters same as the bart, so we set the max_length of query to 20.

#         generated = tokenizer(
#             f"{prompt}", return_tensors="pt", max_length=20
#         ).input_ids.cuda()

#         # perform prediction
#         sample_outputs = model.generate(
#             generated,
#             do_sample=False,
#             top_k=50,
#             max_length=40,
#             top_p=0.90,
#             temperature=0,
#             num_return_sequences=0,
#             pad_token_id=tokenizer.eos_token_id,
#         )
#         # decode the predicted tokens into texts
#         predicted_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
#         predicted_text = predicted_text.strip().replace("\n", "")
#         target = " ".join(predicted_text.split("<|pad|>")[-1].split(" ")[:12])
#         fout.write(target + "\n")
#         fout.flush()
#         # print("predicted_text", predicted_text)


# test()

In [3]:

index = pt.IndexFactory.of(
    "/projects/futhark1/data/wzm289/code/GAR/gar/indices/beir_scifact"
)
print(index.getCollectionStatistics().toString())
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
train_dataset = pt.get_dataset("irds:beir/scifact/train")
train_qrel = train_dataset.get_qrels()
# querys = train_dataset.get_topics("text")
test_dataset = pt.get_dataset("irds:beir/scifact/test")
test_qrel = test_dataset.get_qrels()
test_querys = test_dataset.get_topics("text")

result = pt.Experiment(
    [bm25],
    test_querys,
    test_qrel,
    eval_metrics=["ndcg_cut_10","map"],
    verbose=True,
)

result

Number of documents: 5183
Number of terms: 25508
Number of postings: 442057
Number of fields: 2
Number of tokens: 745811
Field names: [text, title]
Positions:   false



pt.Experiment: 100%|██████████| 1/1 [00:08<00:00,  8.69s/system]


,name,ndcg_cut_10,map
0,BR(BM25),0.683904,0.637799


In [4]:
target_list = []
with open("result/gpt_predict_NDCG.txt","r") as fin:
    for e,line in enumerate(fin):
        target_list.append(" ".join(line.split("<|pad|>")[-1].split(" ")[:7]))
target_list

[' Nanotechnologies are emerging platforms that could',
 ' Only a handful of previously identified',
 ' OBJECTIVES To carry out a further',
 ' CONTEXT One key target of the',
 ' BACKGROUND Lowering serum homocysteine levels with',
 ' BACKGROUND The heritable haemoglobinopathy alpha(\n',
 ' Detailed information on the estimated annual',
 ' Genome-wide association studies are now identifying',
 ' It is established that the immune',
 ' Specifically, the CD4(+)CD25(+) regulatory region has',
 ' Nonetheless, the therapeutic efficacy of activator-human',
 ' Fibrosis is a pathological result of',
 ' Anatomical switches in neuronal cells are',
 ' Anatomical switches in neuronal cells are',
 ' Genome-wide association study (GW2) identified a',
 ' Anator-inhibitor (A-I-cancel-1daq) pairs are non-\n',
 ' To characterize the properties\n',
 ' Unfortunately, the use of non-steroidal anti-inflammatory',
 ' How oncogenic signalling coordinates glycolysis and',
 ' Opposite-sex cells (ISCs) are among t

In [5]:
import pandas as pd 
from gensim.utils import tokenize as gensim_tokenize
from nltk.corpus import stopwords
# print(stopwords.words('english'))
# get test qids
df = pd.read_csv("/projects/futhark1/data/wzm289/code/RLSeq2SeqPytorch/scifact/test.source_rl",names=['qid','query'],sep = "\t")
df = df.astype({"qid": str})
def clean(row):
    text = row["query"].strip().lower()
    tokens = list(gensim_tokenize(text))
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    text = " ".join(tokens)
    return text
df_expand = pd.DataFrame({"qid":df['qid'].to_list(),"query":target_list})
df_expand["query"] = df_expand.apply(clean, axis=1)

target_dict = dict(zip(df_expand['qid'].to_list(),df_expand['query'].to_list()))
df_expand["query"].to_list()

['nanotechnologies emerging platforms could',
 'handful previously identified',
 'objectives carry',
 'context one key target',
 'background lowering serum homocysteine levels',
 'background heritable haemoglobinopathy alpha',
 'detailed information estimated annual',
 'genome wide association studies identifying',
 'established immune',
 'specifically cd cd regulatory region',
 'nonetheless therapeutic efficacy activator human',
 'fibrosis pathological result',
 'anatomical switches neuronal cells',
 'anatomical switches neuronal cells',
 'genome wide association study gw identified',
 'anator inhibitor cancel daq pairs non',
 'characterize properties',
 'unfortunately use non steroidal anti inflammatory',
 'oncogenic signalling coordinates glycolysis',
 'opposite sex cells iscs among',
 'angiotensin converting enzyme ace inhibitors',
 'context bioterrorist attacks involving letters',
 'anandamide endogenous ligand central',
 'background human immunodeficiency virus hiv infection',
 '

In [6]:
def rewrite_query(q):
    return q["query"] + " " + target_dict[q["qid"]]

bm25_ndcg = pt.apply.query(rewrite_query) >> pt.BatchRetrieve(index, wmodel="BM25")

In [7]:
df_expand = pd.DataFrame({"qid":df['qid'].to_list(),"query":target_list})
df_expand["query"] = df_expand.apply(clean, axis=1)
df_expand["query"] = df_expand["query"] + " "+ test_querys['query']


In [8]:
result = pt.Experiment(
    [bm25,bm25_ndcg],
    test_querys,
    test_qrel,
    eval_metrics=["ndcg_cut_10","recall_100","map"],
    verbose=True,
    baseline = 0,
)
result

pt.Experiment: 100%|██████████| 2/2 [00:14<00:00,  7.12s/system]


,name,map,recall_100,ndcg_cut_10,map +,map -,map p-value,recall_100 +,recall_100 -,recall_100 p-value,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,BR(BM25),0.637799,0.926000,0.683904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Compose(pt.apply.??(), BR(BM25))",0.670931,0.918556,0.706004,56.0,71.0,0.013821,4.0,8.0,0.442993,45.0,45.0,0.07837


In [9]:
original_result = pt.Experiment(
    [bm25],
    test_querys,
    test_qrel,
    eval_metrics=["ndcg_cut_10","recall_100","map"],
    verbose=True,
    perquery = True,
)
map_list_1 = []
ndcg_list_1 = []

for item in original_result["qid"].sort_values().unique():
    scores = original_result[original_result.qid == item]['value'].to_list()
    map_list_1.append(scores[0])
    ndcg_list_1.append(scores[2])

pt.Experiment: 100%|██████████| 1/1 [00:06<00:00,  6.77s/system]


In [10]:
ndcg_result = pt.Experiment(
    [bm25_ndcg],
    test_querys,
    test_qrel,
    eval_metrics=["ndcg_cut_10","recall_100","map"],
    verbose=True,
    perquery = True,
)

map_list_2 = []
ndcg_list_2 = []

for item in ndcg_result["qid"].sort_values().unique():
    scores = ndcg_result[ndcg_result.qid == item]['value'].to_list()
    map_list_2.append(scores[0])
    ndcg_list_2.append(scores[2])

pt.Experiment: 100%|██████████| 1/1 [00:07<00:00,  7.23s/system]


In [11]:
# ttest
import scipy.stats
print(scipy.stats.ttest_rel(map_list_1, map_list_2))

Ttest_relResult(statistic=-2.476509005299738, pvalue=0.013820595800838083)


In [12]:
print(scipy.stats.ttest_rel(ndcg_list_1, ndcg_list_2))

Ttest_relResult(statistic=-1.766273414298618, pvalue=0.07837035634252984)


In [11]:
# search_result = bm25.transform(df_expand)[["qid","docno","rank","score"]]

In [15]:
# search_result["q0"] = ["q0"]*len(search_result)
# search_result["run_name"] = ["ndcg@10_reward"]*len(search_result)
# search_result[["qid","q0","docno","rank","score","run_name"]].to_csv("result_run/ndcg_cut_10_run.txt",index = None, header = None,sep = "\t")

In [68]:
# bm25_rm3 = bm25 >> pt.rewrite.RM3(index) >> bm25
# result = pt.Experiment(
#     [bm25_rm3],
#     df,
#     test_qrel,
#     eval_metrics=["ndcg_cut_10","recall_100","map"],
#     verbose=True,
# )
# result

In [8]:
target_list = []
with open("scifact_results_map/gpt_predict.txt","r") as fin:
    for e,line in enumerate(fin):
        target_list.append(" ".join(line.split("<|pad|>")[-1].split(" ")[:7]))
target_list

df_expand = pd.DataFrame({"qid":df['qid'].to_list(),"query":target_list})
df_expand["query"] = df_expand.apply(clean, axis=1)
target_dict = dict(zip(df_expand['qid'].to_list(),df_expand['query'].to_list()))
df_expand["query"].to_list()

['nanotechnologies emerging platforms could',
 'background functional genomic basis',
 'objectives carry',
 'context one key target',
 'background lowering serum homocysteine levels',
 'background heritable haemoglobinopathy alpha',
 'background transmission dengue viruses denv',
 'dicer domain containing ketocholesterol dependent kinase dmk interacts',
 'compared expression',
 'background hepatitis c virus hcv',
 'background emerging evidence suggests association',
 'fibrosis pathological result',
 'efforts develop drugs alzheimer',
 'efforts develop drugs alzheimer',
 'gliomas arising brainstem',
 'activator inhibitor interactions aces spinal circuits',
 'half world population chronically',
 'past two decades',
 'oncogenic signalling coordinates glycolysis',
 'haploid cells amenable genetic',
 'angiotensin converting enzyme ace inhibitors',
 'context bioterrorist attacks involving letters',
 'background bacterial meningitis bs characterized',
 'background human immunodeficiency virus

In [9]:
def rewrite_query(q):
    return q["query"] + " " + target_dict[q["qid"]]
bm25_map = pt.apply.query(rewrite_query) >> pt.BatchRetrieve(index, wmodel="BM25")

In [10]:
df_expand["query"] = df_expand["query"] + " " + test_querys['query']


result = pt.Experiment(
    [bm25,bm25_map],
    test_querys,
    test_qrel,
    eval_metrics=["ndcg_cut_10","recall_100","map"],
    verbose=True,
    baseline = 0,
)
result

pt.Experiment: 100%|██████████| 2/2 [00:14<00:00,  7.16s/system]


,name,map,recall_100,ndcg_cut_10,map +,map -,map p-value,recall_100 +,recall_100 -,recall_100 p-value,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,BR(BM25),0.637799,0.926,0.683904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Compose(pt.apply.??(), BR(BM25))",0.662268,0.924,0.700510,64.0,73.0,0.113598,8.0,10.0,0.875951,54.0,49.0,0.24823
